# 5.6 최근접 이웃(KNN) 분류 모델

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('stroke-standard.csv')
df.shape

(3915, 16)

In [3]:
data = df.drop(['stroke'], axis=1)   # 타겟변수를 제외한 변수만 data에 저장
target = df['stroke']                # 타겟변수만 target에 저장

from imblearn.under_sampling import RandomUnderSampler    # RandomUnderSampler를 import
undersample = RandomUnderSampler(sampling_strategy=0.333, random_state=2) 
                                                          # 타겟변수의 소수 클래스 및 다수 클래스를
                                                          # 1:3의 비율(=1/3)로 언더샘플링
data_under, target_under = undersample.fit_resample(data, target)
                                                          # data 및 target에 언더샘플링 적용

In [4]:
# 50:50 데이터 분할
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
    data_under, target_under, test_size=0.5, random_state=42, stratify=target_under)

print("X_train shape:", X_train.shape) 
print("X_test shape:", X_test.shape) 

X_train shape: (386, 15)
X_test shape: (386, 15)


In [5]:
# 참조 코딩
print(X_train)

           age  avg_glucose_level       bmi  hypertension  heart_disease  \
3447  1.599480          -0.096170  0.255075             1              0   
3687 -0.748656          -0.223311  0.543718             0              0   
2462 -0.977743          -0.376533  0.327236             0              0   
3904  0.511319          -0.224010  0.081889             1              0   
124   1.771295          -0.713713 -0.769608             1              0   
...        ...                ...       ...           ...            ...   
2485 -0.175940          -0.267554  1.770451             0              0   
3434 -0.405027          -0.766107 -0.495397             0              0   
157   1.771295          -0.571436 -1.000522             0              0   
2009 -1.665002           0.378864 -0.091297             0              0   
3507 -0.405027          -0.480388  0.038593             0              0   

      ever_married_encoded  residence_type_encoded  gender_encoded_1.0  \
3447         

In [7]:
# KNN 모델 (Default 모델 with n_neighbors=3)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf_knn = KNeighborsClassifier(n_neighbors=3) # random_state 파라미터가 없음에 주의!
clf_knn.fit(X_train, y_train)
pred = clf_knn.predict(X_test)   # 학습된 Classifier로 테스트 데이터셋 자료이용해서 타겟변수 예측값 생성
accuracy = accuracy_score(y_test, pred)

print ("KNN Training set score:{:.5f}".format(clf_knn.score(X_train, y_train))) 
print ("KNN Test set score:{:.5f}".format(accuracy_score(y_test, pred)))

KNN Training set score:0.84456
KNN Test set score:0.74352


In [10]:
# KNN 모델 (Default 모델 with n_neighbors=3)
clf_knn = KNeighborsClassifier(n_neighbors=3) # random_state 파라미터가 없음에 주의!

# 그리드 서치 실행
from sklearn.model_selection import GridSearchCV
params = {'n_neighbors': range(3, 31)}

grid_knn = GridSearchCV(clf_knn, param_grid=params, scoring='accuracy', cv=5, n_jobs=-1)
grid_knn.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_knn.best_score_))
print("GridSearchCV best parameter:", (grid_knn.best_params_)) 

GridSearchCV max accuracy:0.76950
GridSearchCV best parameter: {'n_neighbors': 22}


In [11]:
best_clf = grid_knn.best_estimator_
pred = best_clf.predict(X_test)
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

Accuracy on test set:0.74870
